In [2]:
from collections import defaultdict, deque

class CourseScheduler:
    """
    A scheduler to manage courses and their prerequisites, detect circular dependencies,
    find an optimal learning path, and identify opportunities for parallel enrollment.

    Attributes:
        total_courses (int): Total number of courses.
        graph (defaultdict[list]): A directed graph representing course prerequisites.
    """

    def __init__(self, total_courses):
        """
        Initializes the CourseScheduler with a specific number of courses.

        Parameters:
            total_courses (int): The total number of courses available in the scheduler.
        """
        self.graph = defaultdict(list)  # Directed graph where each edge (a, b) represents "a is prerequisite of b"
        self.total_courses = total_courses

    def add_prerequisite(self, course, prereq):
        """
        Adds a prerequisite relationship between two courses.

        Parameters:
            course (int): The course ID that has a prerequisite.
            prereq (int): The prerequisite course ID.
        """
        self.graph[prereq].append(course)

    def detect_cycle(self):
        """
        Detects if there are any circular dependencies among the courses.

        Returns:
            bool: True if a cycle is detected, False otherwise.
        """
        visited = [False] * self.total_courses
        rec_stack = [False] * self.total_courses

        def detect_cycle_util(course):
            """Utility function for cycle detection using DFS."""
            if not visited[course]:
                visited[course] = True
                rec_stack[course] = True

                for neighbor in self.graph[course]:
                    if not visited[neighbor] and detect_cycle_util(neighbor):
                        return True
                    elif rec_stack[neighbor]:
                        return True

            rec_stack[course] = False
            return False

        for course in range(self.total_courses):
            if detect_cycle_util(course):
                return True
        return False

    def find_optimal_path(self):
        """
        Determines an optimal learning path given the prerequisites.

        Returns:
            list: An ordered list of course IDs representing the recommended learning path.
            str: A message indicating if a circular dependency is detected.
        """
        if self.detect_cycle():
            return "Circular dependency detected. Cannot generate a learning path."

        visited = [False] * self.total_courses
        stack = []

        def topological_sort_util(course):
            """Utility function for topological sorting."""
            visited[course] = True
            for i in self.graph[course]:
                if not visited[i]:
                    topological_sort_util(i)
            stack.append(course)

        for i in range(self.total_courses):
            if not visited[i]:
                topological_sort_util(i)

        stack.reverse()  # The reverse of the stack gives the order of courses to be completed.
        return stack

    def find_parallel_courses(self):
        """
        Identifies sets of courses that can be taken in parallel, based on their prerequisites.

        Returns:
            list of lists: Each sublist contains courses that can be enrolled in parallel.
        """
        in_degree = [0] * self.total_courses
        for i in self.graph:
            for j in self.graph[i]:
                in_degree[j] += 1

        queue = deque()
        for i in range(self.total_courses):
            if in_degree[i] == 0:
                queue.append(i)

        parallel_courses = []
        while queue:
            current_level = []
            for _ in range(len(queue)):
                course = queue.popleft()
                current_level.append(course)
                for neighbor in self.graph[course]:
                    in_degree[neighbor] -= 1
                    if in_degree[neighbor] == 0:
                        queue.append(neighbor)
            parallel_courses.append(current_level)

        return parallel_courses

# Example usage:
scheduler = CourseScheduler(5)
scheduler.add_prerequisite(1, 0)
scheduler.add_prerequisite(2, 1)
scheduler.add_prerequisite(3, 2)
scheduler.add_prerequisite(4, 1)
print("Parallel Enrollment Opportunities:", scheduler.find_parallel_courses())


Parallel Enrollment Opportunities: [[0], [1], [2, 4], [3]]
